***This notebook takes speaker data provided by the Accent Dataset, cleans and verifies it, and generates a `.tsv` file which can be passed on to the CorporaCreator. The CorporaCreator further processes the data and generates requisite data for train, test, and dev sets.***

#### CONSTRUCTING TSV FILE

In [ ]:
%cd /content/drive/My Drive/Accent\ Data/recordings/recordings/
!ls >> lsout.csv
!mv lsout.csv /content/drive/My\ Drive/Accent\ Data/

Reading the list into dataframe into `rec_list`, removing duplicates and non audio file entries.

In [ ]:
%cd /content/drive/My\ Drive/Accent\ Data/
rec_list = pd.read_csv('lsout.csv', names=['filename_merge'])
rec_list.drop(rec_list.loc[rec_list['filename_merge']=='lsout.csv'].index, inplace=True)
rec_list.drop_duplicates(inplace=True)

Reading data file for Accent Data, preproceesing -> `file_speakers`

In [ ]:
## THIS SHOULD RUN ONLY ONCE
speakers = pd.read_csv('/content/drive/My Drive/Accent Data/speakers_all.csv')
#csv_read.to_csv('/content/speakers_tsv_all.tsv', sep='\t', index=False)
file_speakers = speakers[speakers['file_missing?']!=True]
file_speakers['filename_merge'] = file_speakers['filename'] + ".mp3"

Merge `rec_list` and `file_speakers`. Drop unnecessary columns.

In [ ]:
merge_audio_data = pd.merge(rec_list, file_speakers, on='filename_merge')

audio_data = merge_audio_data.drop(['age_onset', 'birthplace', 'filename','country', 'file_missing?',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'], axis=1)

Add required columns, rename some columns.

In [ ]:
#Still in Accent Data/

with open('reading-passage.txt', 'r') as file:
    sentence = file.read().replace('\n', '')
audio_data['sentence'] = re.sub(r'[^\w\s]','',sentence)
audio_data['up_votes']=4
audio_data['down_votes']=0
audio_data['locale']="en"
audio_data['bucket']=""
audio_data.rename(columns={"native_language": "accent", "filename_merge": "path", "sex": "gender", "speakerid": "client_id"}, inplace = True)

In [ ]:
audio_data.to_csv('clips.csv', sep='\t', index = False) #Parse into a clips.csv file
audio_data

,path,age,accent,gender,client_id,sentence,up_votes,down_votes
0,afrikaans1.mp3,27.0,afrikaans,female,1,Please call Stella Ask her to bring these thi...,4,0
1,afrikaans2.mp3,40.0,afrikaans,male,2,Please call Stella Ask her to bring these thi...,4,0
2,afrikaans3.mp3,43.0,afrikaans,male,418,Please call Stella Ask her to bring these thi...,4,0
3,afrikaans4.mp3,26.0,afrikaans,male,1159,Please call Stella Ask her to bring these thi...,4,0
4,afrikaans5.mp3,19.0,afrikaans,male,1432,Please call Stella Ask her to bring these thi...,4,0
...,...,...,...,...,...,...,...,...
2133,yoruba3.mp3,46.0,yoruba,female,766,Please call Stella Ask her to bring these thi...,4,0
2134,yoruba4.mp3,46.0,yoruba,male,851,Please call Stella Ask her to bring these thi...,4,0
2135,yoruba5.mp3,47.0,yoruba,female,2023,Please call Stella Ask her to bring these thi...,4,0
2136,yupik1.mp3,31.0,yupik,female,571,Please call Stella Ask her to bring these thi...,4,0


Pass the `clips.csv` file to CorporaCreator to generate required train, test, dev files.

In [ ]:
#In case oonversion from mp3 to wav is necessary

i=c[1]
src = str(i)
dest = str(i).split(".")[0] + ".wav"
sound = AudioSegment.from_mp3(src)
sound.export(dest,format="wav")
print("converting ",src,"to ", dest)
print("Converting ", dest, " to proper format")
w = wave.open(dest, 'r')
rate = w.getframerate()
frames = w.getnframes()
buffer = w.readframes(frames)
print(rate, frames)
#print(model.sampleRate())
type(buffer)
print("done")